In [ ]:
#%% Pandas Milestone Action Items 
import pandas as pd
import matplotlib.pyplot as plt
path = 'C:/Users/Kathleen/Documents/Boot Camp/12 day/practice files/'
ecdata = pd.read_csv(path + 'Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_20231207.csv')

#peruse 
ecdata.info()
summary = ecdata.describe()
ecdata["Period"].value_counts()

####### 
#1. How many different companies are represented in the data set?
len(set(ecdata["Company Name"])) 
#answer = 787 

####### 
#2. What is the total number of jobs created for businesses in Queens? 
#answer = 1196

# a. For context, how many rows of data from queens? 
ecdata.value_counts("Borough")

#b: filter the data for queens -- fails... 
ecdata.loc[ecdata["Borough"] == "QUEENS", ["Period", "Company Name", "Job created"]]
ecdata.loc[ecdata["Borough"] == "QUEENS", ["Job created"]].sum()

#c total jobs created for each borough 
ecdata.groupby(["Borough"])["Job created"].sum()

#d total jobs created for just queens 
ecdata.groupby(["Borough"]).get_group("QUEENS")["Job created"].sum()

#######
#3. How many different unique email domains names are there in the data set?
#answer = 589 

#find the email data
ecdata.columns
ecdata.info()

#make a function to convert strings to lower (not all values are strings, some are na)
def ifstringlower(datapoint): 
    if isinstance(datapoint, str):
        return datapoint.lower()
    else:
        return datapoint
#apply the function 
ecdata["company email"] = ecdata["company email"].apply(ifstringlower) 

#separate the series
emails = ecdata["company email"] 

#use split to parse on the @ sign into two columns 
emails_split = emails.str.split('@', n=1, expand = True).rename(columns = {0: "user", 1: "domain"})
emails_split.head()

#count unique values 
emails_split["domain"].dropna().nunique() #the pandas way is nunique
len(set(emails_split["domain"].dropna())) #double check  with the set method. 


#sidebar random graphing
#these numbers are huge so let's look at a graph for a sanity check. 
plt.ticklabel_format(style = 'sci')
fig = summary["Total Savings"].sort_values().plot()
fig.set_yscale('log')
plt.show() 

####### 
#4 Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?

#make col name easier to work with 
ecdata.columns
ecdata.info()
ecdata.rename(columns={"Neighborhood Tabulation Area (NTA) (2020)" : "NTA"}, inplace=True)

#count businesses by NTA
#ntacounts = ecdata["NTA"].value_counts(["NTA"]) #for some reason if you restate the variable in parens here it normalizes. (??) 
ntacounts = ecdata["NTA"].value_counts()

#sum jobs created 
jobsum = ecdata[["Job created", "NTA"]].groupby(["NTA"]).sum() 

#mean savings 
meansaved = ecdata[["Total Savings", "NTA"]].groupby(["NTA"]).mean()

#concatenate (cbind) my 3 series 
summary = pd.concat([ntacounts, meansaved, jobsum], axis = 1)

#output #filter with loc 
output = summary.loc[summary["count"] >= 5, ['count', 'Job created', 'Total Savings']]
#this syntax and i ar enemies. dataframe.loc[bracket RENAMEDATAFRAME [bracket "columname" ] >=5 conditional comma, [list of output columns] ]

path = 'C:/Users/Kathleen/Documents/Boot Camp/12 day/practice files/'
output.to_csv(path+'day4milestone.csv')


In [ ]:
#%% Plotting  Milestone action items 

import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.dates as mdates
from matplotlib.dates import (YEARLY, DateFormatter, RRuleLocator, drange,
                              rrulewrapper)
path = 'C:/Users/Kathleen/Documents/Boot Camp/12 day/practice files/'
ecdata = pd.read_csv(path + 'Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_20231207.csv')

#peruse 
ecdata.info()
summary = ecdata.describe()
ecdata["Period"].value_counts()
ecdata.columns

#1 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 3))
ax1.scatter(ecdata['Job created'], ecdata['Total Savings'])
ax1.set_title('Total Savings by Jobs Created (linear scale)')
ax1.set_xlabel('Jobs created (by business)')
ax1.set_ylabel('Total savings (by biz)')
ax2.scatter(ecdata['Job created'], ecdata['Total Savings'])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_title('Total Savings by Jobs Created (log scale)')
ax2.set_xlabel('Jobs created (by business)')
ax2.set_ylabel('Total savings (by biz)')


#2 histogram of the log of the total savings 
ecdata['Total Savings'].describe()
fig, ax = plt.subplots()
ax.hist(ecdata['Total Savings'], bins = 30)
ax.set_xscale('log')
ax.set_yscale('log')


#3 
#conver the effective date in to an actual posixct / datetime variable 
#group by month 
#graph the summary 
#going by  Period doesn't work because not quant data can't be line graphed
summary = ecdata.groupby(["Period"])["Job created"].sum()
fig, ax1 = plt.subplots()
ax1.plot(summary.index, summary)
summary.index
ecdata['Effective Date'].value_counts()

#create a time variable 
ecdata["Effective Date"].head(20)
print(help(pd.to_datetime))
ecdata["effdate"] = pd.to_datetime(ecdata['Effective Date'], dayfirst=False)
ecdata.info()
ecdata["effdate"].head(20)

#this method groups by month across years, (or vv) so not exactly what we want. 
ecdata.groupby(ecdata['effdate'].dt.month)["Total Savings"].sum()
monthsum = ecdata.groupby(ecdata['effdate'].dt.month)["Total Savings"].sum()


#group the savings from each month 
monthsavings = ecdata.groupby(pd.Grouper(key="effdate", freq="1M"))["Total Savings"].sum()


#sanity checks: 
monthsavings.sum()
ecdata["Total Savings"].sum()

monthsavings.loc['September, 2021']
september_21 = ecdata[ecdata["effdate"].between('2021/09/01', '2021/10/1')]
september_21["Total Savings"].sum()
 


#graph the result 
fig, ax = plt.subplots(figsize = (10, 3))
ax.plot(monthsavings.index, monthsavings) 
ax.xaxis.set_tick_params(rotation=30, labelsize =10)

#This is actually works! graphs values by months, labels by years! But here we go... 
rule = rrulewrapper(YEARLY)
loc = RRuleLocator(rule)
formatter=DateFormatter('%Y')

fig, ax = plt.subplots(figsize = (10, 3))
plt.plot(monthsavings.index, monthsavings, 'o') 
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_tick_params(rotation=30, labelsize =10)


##alternate approach: plot by month 
fig, ax = plt.subplots(figsize = (10, 3))
ax.plot(monthsum.index, monthsum) 
ax.set_xticks(np.arange(1,13,1), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.xaxis.set_tick_params(rotation=30, labelsize =10)
#cyclical - shows most savings are reported at end of fiscal year.


#For table display only! 
#if you make this change before you make the graph, you're making your indx text, 
# and you won't be able to use date formatter. 
# make the index be the name of the month, year
monthsavings.index = monthsavings.index.strftime('%B, %Y')
monthsavings
